<a href="https://colab.research.google.com/github/chrispydocodes/Prediction-of-Project-Sales/blob/main/Project_1_Chris_Do.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Prediction of Product Sales
### Author: Chris Do

# Project Overview

- This project serves as a sales prediction for food items sold at various stores. 
- The goal of this is to help the retailer understand the properties of products and outlets that play crucial roles in increasing sales.

# Load and Inspect Data

- Download the data [here](https://drive.google.com/file/d/1syH81TVrbBsdymLT_jl2JIf6IjPXtSQw/view), then import the necessary libraries in order to load and manipulate it.

First, we're going to import Pandas in order to load the data as a DataFrame.

In [1]:
import pandas as pd

Next, we define our data's file path as `filename`.

In [2]:
filename = '/content/sales_predictions.csv'

Then, we create a DataFrame named `df`.

In [3]:
df = pd.read_csv(filename)

Now, we will view our DataFrame's info and first 5 entries.

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8523 entries, 0 to 8522
Data columns (total 12 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Item_Identifier            8523 non-null   object 
 1   Item_Weight                7060 non-null   float64
 2   Item_Fat_Content           8523 non-null   object 
 3   Item_Visibility            8523 non-null   float64
 4   Item_Type                  8523 non-null   object 
 5   Item_MRP                   8523 non-null   float64
 6   Outlet_Identifier          8523 non-null   object 
 7   Outlet_Establishment_Year  8523 non-null   int64  
 8   Outlet_Size                6113 non-null   object 
 9   Outlet_Location_Type       8523 non-null   object 
 10  Outlet_Type                8523 non-null   object 
 11  Item_Outlet_Sales          8523 non-null   float64
dtypes: float64(4), int64(1), object(7)
memory usage: 799.2+ KB


In [5]:
df.head()

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
0,FDA15,9.30,Low Fat,0.016047,Dairy,249.8092,OUT049,1999,Medium,Tier 1,Supermarket Type1,3735.1380
1,DRC01,5.92,Regular,0.019278,Soft Drinks,48.2692,OUT018,2009,Medium,Tier 3,Supermarket Type2,443.4228
2,FDN15,17.50,Low Fat,0.016760,Meat,141.6180,OUT049,1999,Medium,Tier 1,Supermarket Type1,2097.2700
3,FDX07,19.20,Regular,0.000000,Fruits and Vegetables,182.0950,OUT010,1998,NaN,Tier 3,Grocery Store,732.3800
4,NCD19,8.93,Low Fat,0.000000,Household,53.8614,OUT013,1987,High,Tier 3,Supermarket Type1,994.7052


# Clean Data

Let's go through our data and clean it up so we have a better time understanding and using it.

## Data Dictionary

- `Item_Identifier`:	Unique product ID
- `Item_Weight`:	Weight of product
- `Item_Fat_Content`:	Whether the product is low fat or regular
- `Item_Visibility`: The percentage of total display area of all products in a store allocated to the particular product
- `Item_Type`:	The category to which the product belongs
- `Item_MRP`:	Maximum Retail Price (list price) of the product
- `Outlet_Identifier`:	Unique store ID
- `Outlet_Establishment_Year`:	The year in which store was established
- `Outlet_Size`:	The size of the store in terms of ground area covered
- `Outlet_Location_Type`:	The type of area in which the store is located
- `Outlet_Type`:	Whether the outlet is a grocery store or some sort of supermarket
- `Item_Outlet_Sales`:	Sales of the product in the particular store. This is the target variable to be predicted.

## Rows and Columns


Adding the `.shape` attribute in front of out DataFrame tells us its number of dimensions, as well as the number of items for each dimension.

In [6]:
df.shape

(8523, 12)

- Our DataFrame has 8523 rows (entries) under 12 columns

## Variable Datatypes

We can use the `.dtypes` attribute to see the datatype of each variable in our DataFrame.

In [7]:
df.dtypes

Item_Identifier               object
Item_Weight                  float64
Item_Fat_Content              object
Item_Visibility              float64
Item_Type                     object
Item_MRP                     float64
Outlet_Identifier             object
Outlet_Establishment_Year      int64
Outlet_Size                   object
Outlet_Location_Type          object
Outlet_Type                   object
Item_Outlet_Sales            float64
dtype: object

- Our DataFrame includes objects, floats, and integers as variables.

## Duplicate Entries

The `.duplicated()` method will check for duplicate values in our DataFrame, returning either `True` or `False` for each entry. We can also use the `.sum()` method to return the total number of `True` (duplicate) values rather than checking each row ourselves.

In [8]:
df.duplicated().sum()

0

- Thankfully, we don't have to worry about any duplicate entries in our data.

## Missing Values

The `.isna()` method checks for null/missing values in each entry. Again, we will add `.sum()` to return a total number of missing values.

In [9]:
df.isna().sum()

Item_Identifier                 0
Item_Weight                  1463
Item_Fat_Content                0
Item_Visibility                 0
Item_Type                       0
Item_MRP                        0
Outlet_Identifier               0
Outlet_Establishment_Year       0
Outlet_Size                  2410
Outlet_Location_Type            0
Outlet_Type                     0
Item_Outlet_Sales               0
dtype: int64

- Our data is missing 1463 `Item Weight` values and 2410 `Outlet Size` values


## Addressing Missing Values

- Item weights can't really be filled in with an estimated value, since an item's weight has no correlation with the other items sold in a store.
  - We also shouldn't drop the entries just for missing weights, since a large percentage of our item weights are missing.
- Outlet sizes could also be important in determining sales of a product.
- I will fill in missing values with 'Missing'.

The `.fillna()` method fills in missing values. Adding the `inplace=True` argument overwrites our DataFrame.

In [10]:
df.fillna('Missing', inplace=True)

## Double Checking for Missing Values

In [11]:
df.isna().sum()

Item_Identifier              0
Item_Weight                  0
Item_Fat_Content             0
Item_Visibility              0
Item_Type                    0
Item_MRP                     0
Outlet_Identifier            0
Outlet_Establishment_Year    0
Outlet_Size                  0
Outlet_Location_Type         0
Outlet_Type                  0
Item_Outlet_Sales            0
dtype: int64

We have no missing values left in our data.

## Inconsistent Categories/Entries

Let's check our datatype columns for inconsistent entries.

First, we get a list of our column datatypes.

In [12]:
df.dtypes

Item_Identifier               object
Item_Weight                   object
Item_Fat_Content              object
Item_Visibility              float64
Item_Type                     object
Item_MRP                     float64
Outlet_Identifier             object
Outlet_Establishment_Year      int64
Outlet_Size                   object
Outlet_Location_Type          object
Outlet_Type                   object
Item_Outlet_Sales            float64
dtype: object

The `.value_counts()` method returns the unique values of each column.

### Item Fat Content

In [13]:
df['Item_Fat_Content'].value_counts()

Low Fat    5089
Regular    2889
LF          316
reg         117
low fat     112
Name: Item_Fat_Content, dtype: int64

There should only be `Low Fat` or `Regular` values, so we'll replace `LF`,`reg`, and `low fat` using the `.replace()` method.

In [19]:
df['Item_Fat_Content'].replace('LF','Low Fat',inplace=True)
df['Item_Fat_Content'].replace('low fat','Low Fat',inplace=True)
df['Item_Fat_Content'].replace('reg','Regular',inplace=True)
df['Item_Fat_Content'].value_counts()


Low Fat    5517
Regular    3006
Name: Item_Fat_Content, dtype: int64

### Item Type

In [20]:
df['Item_Type'].value_counts()

Fruits and Vegetables    1232
Snack Foods              1200
Household                 910
Frozen Foods              856
Dairy                     682
Canned                    649
Baking Goods              648
Health and Hygiene        520
Soft Drinks               445
Meat                      425
Breads                    251
Hard Drinks               214
Others                    169
Starchy Foods             148
Breakfast                 110
Seafood                    64
Name: Item_Type, dtype: int64

### Outlet Size

In [22]:
df['Outlet_Size'].value_counts()

Medium     2793
Missing    2410
Small      2388
High        932
Name: Outlet_Size, dtype: int64

### Outlet Location Type

In [23]:
df['Outlet_Location_Type'].value_counts()

Tier 3    3350
Tier 2    2785
Tier 1    2388
Name: Outlet_Location_Type, dtype: int64

### Outlet Type

In [24]:
df['Outlet_Type'].value_counts()

Supermarket Type1    5577
Grocery Store        1083
Supermarket Type3     935
Supermarket Type2     928
Name: Outlet_Type, dtype: int64

## Summary Statistics of Numerical Columns


In [25]:
df.describe(include = 'all').round(4)

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
count,8523,8523,8523,8523.0000,8523,8523.0000,8523,8523.0000,8523,8523,8523,8523.0000
unique,1559,416,2,NaN,16,NaN,10,NaN,4,3,4,NaN
top,FDW13,Missing,Low Fat,NaN,Fruits and Vegetables,NaN,OUT027,NaN,Medium,Tier 3,Supermarket Type1,NaN
freq,10,1463,5517,NaN,1232,NaN,935,NaN,2793,3350,5577,NaN
mean,NaN,NaN,NaN,0.0661,NaN,140.9928,NaN,1997.8319,NaN,NaN,NaN,2181.2889
std,NaN,NaN,NaN,0.0516,NaN,62.2751,NaN,8.3718,NaN,NaN,NaN,1706.4996
min,NaN,NaN,NaN,0.0000,NaN,31.2900,NaN,1985.0000,NaN,NaN,NaN,33.2900
25%,NaN,NaN,NaN,0.0270,NaN,93.8265,NaN,1987.0000,NaN,NaN,NaN,834.2474
50%,NaN,NaN,NaN,0.0539,NaN,143.0128,NaN,1999.0000,NaN,NaN,NaN,1794.3310
75%,NaN,NaN,NaN,0.0946,NaN,185.6437,NaN,2004.0000,NaN,NaN,NaN,3101.2964


# Exploratory Data Analysis

# Explanatory Data Analysis